# Classification Dataset

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [1]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import ClusterCentroids
from scipy.stats import ks_2samp
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import seaborn as sb
from IPython.display import display

RANDOM_STATE = 42  

In [2]:
dataset = pd.read_csv('../../data/complete_dataset_no_outliers.csv', dtype={'cyclist_team': str})
dataset['label'] = (dataset['raw_position'] <= 20).astype(int) #1 if top 20, 0 otherwise
dataset.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568810 entries, 0 to 568809
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   cyclist_url                  568810 non-null  object 
 1   cyclist_name                 568810 non-null  object 
 2   birth_year                   568810 non-null  int64  
 3   weight                       459078 non-null  float64
 4   height                       459078 non-null  float64
 5   nationality                  568810 non-null  object 
 6   race_url                     568810 non-null  object 
 7   race_name                    568810 non-null  object 
 8   points                       568810 non-null  float64
 9   uci_points                   401255 non-null  float64
 10  length                       568810 non-null  float64
 11  climb_total                  454409 non-null  float64
 12  profile                      454409 non-null  float64
 13 

Feature da eliminare:

- cyclist_url (non generalizzabili)
- cyclist_name (non generalizzabili)
- birth_year (gia incluso in age)
- nationality (troppo sbilanciamento nel dataset)
- weight, height (gia incluso in BMI)
- uci_points (troppo pochi e gia inclusi in points)
- race_url (non generalizzabili)
- race_name (non generalizzabili)
- climb_total, profile (gia inclusi in race_physical_effort)
- mostly_tarmac (sbilanciato)
- cyclist_team (non generalizzabili)
- date (non generalizzabili)
- raw_position, position (obv)
- delta (obv) 
- race_year (non generalizzabili)
- race_stage (non generalizzabili)
- cyclist_age_group (??)
- cyclist_climb_power (feature generata con delta, obv)
- points, startlist_quality (gia inclusti in race_prestige)

We must choose between cyclist_age and cyclist_age_group as both these features represent the same information bu give different advantage/disadvantage to our classification process: 
- cyclist_age -> lower dimensionality (no one-hot) but unbalanced representation
- cyclist_age_group -> arbitraty encoding but balanced representation 

we chose to delete cyclist_age as the encoding is arbitraty but it still respects the "order" of the age_groups

In [3]:
columns = ['length', 'race_season', 'cyclist_bmi', 'cyclist_age_group', 'climb_percentage', 'race_physical_effort', 'race_prestige', 'race_year', 
           'previous_mean_position', 'previous_mean_delta', 'previous_mean_cp', 'cyclist_previous_experience', 'num_participants', 'label']
dataset = dataset[columns].dropna()

In [4]:
development_dataset = dataset[dataset['race_year'] < 2022]
test_dataset = dataset[dataset['race_year'] >= 2022]

development_set = development_dataset.drop(columns=['race_year'])
developmente_label = development_set.pop('label')
test_set = test_dataset.drop(columns=['race_year'])
test_label = test_set.pop('label')

In [5]:
developmente_label.value_counts(normalize=True)

label
0    0.86389
1    0.13611
Name: proportion, dtype: float64

In [6]:
test_label.value_counts(normalize=True)

label
0    0.871362
1    0.128638
Name: proportion, dtype: float64

In [7]:
def discretize_data(dataset, variables):
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable] = dataset[variable].map(mapping).astype(int)
    return dataset

In [8]:
variables = ['cyclist_age_group']
development_set = discretize_data(development_set, variables)
cat_columns = ['race_season']
development_set = pd.get_dummies(development_set, columns = cat_columns, prefix_sep='%')
development_set

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
5,135.0,22.256908,1,0.003244,0.006662,0.193636,0.823060,161.333333,0.000118,0.024713,109,False,True,False,False
6,100.0,22.256908,1,0.003980,0.004484,0.193636,0.780877,138.285714,0.001042,0.049022,109,False,True,False,False
7,199.0,22.256908,1,0.009281,0.041416,0.193636,0.738174,121.000000,0.003588,0.071902,108,False,False,True,False
8,244.0,22.256908,1,0.006988,0.140635,0.193636,0.739753,128.111111,0.002734,0.097660,106,False,False,True,False
9,244.0,22.256908,1,0.009520,0.063869,0.193636,0.743873,115.300000,0.003841,0.131921,106,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568804,142.0,20.830818,1,0.006106,0.027745,0.074522,0.590384,510.512500,0.001754,0.146172,157,False,True,False,False
568805,146.0,20.830818,1,0.015171,0.072880,0.074522,0.594460,514.234568,0.001715,0.147058,152,False,True,False,False
568806,228.0,20.830818,1,0.010531,0.123371,0.074522,0.598033,520.695122,0.001689,0.148332,152,False,True,False,False
568807,210.0,20.830818,1,0.000814,0.004045,0.074522,0.591069,514.421687,0.001688,0.150036,151,False,True,False,False


In [9]:
def ks_test(undersampled_dataset, dataset):    
    columns = undersampled_dataset.columns
    test_results = [
        ks_2samp(
            dataset[column],
            undersampled_dataset[column],
            alternative="two-sided"
        )
        for column in columns if column != 'label'
    ]
    test_data = [(
        test.statistic,
        test.pvalue,
        test.statistic_location
        )
        for test in test_results        
    ]
    test_data = pd.DataFrame(test_data, columns=["KS_test", "p_value", "margin"])

    return test_data.describe()

In [10]:
scv = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
ks_tests = []

for train_index, val_index in scv.split(development_set, developmente_label):
    X_train, X_val = development_set.iloc[train_index], development_set.iloc[val_index]
    y_train, y_val = developmente_label.iloc[train_index], developmente_label.iloc[val_index]

    rus = RandomUnderSampler(random_state=RANDOM_STATE)
    X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)

    undersampled_train_set = pd.DataFrame(numpy.hstack((X_resampled, y_resampled)), columns=(list(X_train.columns) + ['label']))

    ks_tests.append(ks_test(undersampled_train_set, X_train))
    

In [11]:
for i, test in enumerate(ks_tests, start=1):
    print(f"KS_Test Fold {i}:")
    display(test)
    print("\n")

KS_Test Fold 1:


,KS_test,p_value
count,15.000000,1.500000e+01
mean,0.023958,2.778977e-01
std,0.031049,4.042904e-01
min,0.000937,0.000000e+00
25%,0.003624,1.829873e-45
50%,0.005977,3.258153e-02
75%,0.030306,4.805646e-01
max,0.101284,1.000000e+00




KS_Test Fold 2:


,KS_test,p_value
count,15.000000,1.500000e+01
mean,0.023557,4.000825e-01
std,0.031364,4.870824e-01
min,0.000200,0.000000e+00
25%,0.001914,1.052407e-45
50%,0.006016,3.084225e-02
75%,0.031442,9.724797e-01
max,0.101556,1.000000e+00




KS_Test Fold 3:


,KS_test,p_value
count,15.000000,1.500000e+01
mean,0.023626,3.721714e-01
std,0.031865,4.536079e-01
min,0.000145,0.000000e+00
25%,0.002017,1.744993e-45
50%,0.004416,2.108211e-01
75%,0.030771,9.473214e-01
max,0.102942,1.000000e+00




KS_Test Fold 4:


,KS_test,p_value
count,15.000000,1.500000e+01
mean,0.023470,4.162508e-01
std,0.031833,4.657754e-01
min,0.000386,0.000000e+00
25%,0.002032,6.247522e-47
50%,0.005342,7.457603e-02
75%,0.031035,9.346825e-01
max,0.102609,1.000000e+00




KS_Test Fold 5:


,KS_test,p_value
count,15.000000,1.500000e+01
mean,0.024104,3.325296e-01
std,0.031884,4.380376e-01
min,0.000264,0.000000e+00
25%,0.002704,1.640968e-43
50%,0.005782,4.242990e-02
75%,0.030833,7.850344e-01
max,0.102562,1.000000e+00


In [12]:
test_set['label'] = test_label
test_set = discretize_data(test_set, variables)
cat_columns = ['race_season']
test_set_encoded = pd.get_dummies(test_set, columns = cat_columns, prefix_sep='%')
test_set_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,label,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
196,203.9,21.705129,3,0.009858,0.092363,0.225487,0.729362,470.333333,0.000047,0.009954,111,0,False,False,False,True
197,254.1,21.705129,3,0.013617,0.198139,0.248896,0.666310,458.285714,0.000047,0.012297,126,0,False,True,False,False
199,183.7,20.756387,0,0.008977,0.068267,0.228368,0.828177,559.000000,0.000032,0.006341,108,0,False,False,False,True
431,123.4,18.827160,3,0.018476,0.158518,0.000000,0.454167,211.500000,0.000211,0.000813,116,0,False,False,False,True
432,164.6,18.827160,3,0.012704,0.116349,0.051346,0.424452,156.250000,0.000243,0.001490,172,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568453,218.0,20.305175,4,0.011482,0.184458,0.078556,0.416825,464.256065,0.002305,0.598772,170,0,False,False,False,True
568454,165.6,20.305175,4,0.022234,0.343537,0.078556,0.417961,466.758065,0.002295,0.599307,163,0,False,False,False,True
568455,193.0,20.305175,4,0.018705,0.235529,0.078556,0.419099,469.501340,0.002286,0.600163,153,0,False,False,False,True
568456,154.0,20.305175,4,0.010779,0.028805,0.078556,0.420047,468.336898,0.002282,0.600797,147,0,False,False,False,True


Abbiamo trasformato gli attributi categorici in equivalenti numerici per poter essere processati dai modelli di learning: 
- one-hot encoding: è stato utilizzato quando l'attributo categorico non è di tipo ordinale, in questo caso non siamo interessati a preservare ordine o relazioni di vicinanza tra i valori di questo attributo e pertanto vogliamo massimizzare la dissimilarità tra le classi
- discretizzazione: abbiamo usato questa tecnica solo nel caso dell'attributo cyclist_age_group, in quanto queste classi possiedono relazioni di ordine e vicinanza (attributo ordinale)

In [13]:
development_set['label'] = developmente_label

In [14]:
development_dataset['label'].value_counts(normalize=True)

label
0    0.86389
1    0.13611
Name: proportion, dtype: float64

In [15]:
development_set.to_csv('../../data/ml_datasets/undersampling/dev_set.csv', index=False)
test_set_encoded.to_csv('../../data/ml_datasets/undersampling/test_set.csv', index=False)